In [1]:
import csv
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OrdinalEncoder

from constants.constants import *

# Embeds graphs in Jupyter notebook (instead of pop-ups)
%matplotlib inline

# pd.set_option('display.max_columns', None) # show all columns

['gk', 'meta', 'squad', 'attack']


# Creating the scores Dataframe

In [2]:
COUNTRIES = ['ENG', 'SPA', 'ITA', 'GER', 'FRA']

In [3]:
individual_score_dfs = {}

for country in COUNTRIES:
    for year in range(2019, 2024):
        try:
            individual_score_dfs[f"{year}_{country}"] = pd.read_csv(
                f"./data/scores/raw/{country}{year}_scores.csv",
                encoding='utf-8'
            )
        except FileNotFoundError as e:
            print(e)

scores_df = pd.concat(individual_score_dfs.values(), ignore_index=True)

In [4]:
dropped_cols = []
individual_dfs = {}

for country in COUNTRIES:
    for year in range(2018, 2023):
        try:
            df = pd.read_csv(
                f"./data/teams/raw/{country}{year}_teams_for_{year+1}.csv",
                encoding='utf-8'
            )
            cols_to_drop = df.columns[df.isnull().all()].tolist()
            df.drop(columns=cols_to_drop, inplace=True)
            individual_dfs[f"{year}_{country}"] = df
            for col in cols_to_drop:
                if col not in dropped_cols:
                    dropped_cols.append(col)
            df["country"] = country
        except FileNotFoundError:
            print(f"File not found for {year}")

if len(individual_dfs.values()) > 0:
    teams_df = pd.concat(
        individual_dfs.values(),
        join="inner",
        ignore_index=True
    )
    print(f"Dropped cols: {dropped_cols}")
else:
    print(f"No dataframes found for any year")

Dropped cols: []


In [5]:
teams_perf = set(teams_df['squad'].values)
teams_scores = set(scores_df['home_team'].values).union(set(scores_df['away_team'].values))

old_names = sorted(list(teams_perf - teams_scores))
new_names = sorted(list(teams_scores - teams_perf))

print(f'{len(old_names)} conflicts: {old_names}')
print(f'{len(new_names)} conflicts: {new_names}')

34 conflicts: ['Ajaccio', 'Alavés', 'Almería', 'Arminia', 'Athletic Club', 'Atlético Madrid', 'Clermont Foot', 'Cádiz', 'Darmstadt 98', 'Düsseldorf', 'Eint Frankfurt', 'Granada', 'Greuther Fürth', 'Hellas Verona', 'Hertha BSC', 'Köln', 'Leeds United', 'Leganés', 'Leicester City', 'Leverkusen', 'Luton Town', "M'Gladbach", 'Mainz 05', 'Milan', 'Newcastle Utd', 'Norwich City', "Nott'ham Forest", 'Nîmes', 'Paderborn 07', 'Paris S-G', 'Roma', 'SPAL', 'Saint-Étienne', 'Schalke 04']
39 conflicts: ['AC Ajaccio', 'AC Milan', 'AS Roma', 'Alaves', 'Almeria', 'Arminia Bielefeld', 'Ath Bilbao', 'Atl. Madrid', 'B. Monchengladbach', 'Bayer Leverkusen', 'Cadiz CF', 'Clermont', 'Darmstadt', 'Dusseldorf', 'Eintracht Frankfurt', 'FC Koln', 'Granada CF', 'Grenoble', 'Greuther Furth', 'Hamburger SV', 'Hertha Berlin', 'Holstein Kiel', 'Leeds', 'Leganes', 'Leicester', 'Luton', 'Mainz', 'Newcastle', 'Nimes', 'Norwich', 'Nottingham', 'PSG', 'Paderborn', 'Paris FC', 'Schalke', 'Sochaux', 'Spal', 'St Etienne', '

In [6]:
name_mapping = {
    'Ajaccio': 'AC Ajaccio',
    'Alavés': 'Alaves',
    'Almería': 'Almeria',
    'Arminia': 'Arminia Bielefeld',
    'Athletic Club': 'Ath Bilbao',
    'Atlético Madrid': 'Atl. Madrid',
    'Clermont Foot': 'Clermont',
    'Cádiz': 'Cadiz CF',
    'Darmstadt 98': 'Darmstadt',
    'Düsseldorf': 'Dusseldorf',
    'Eint Frankfurt': 'Eintracht Frankfurt',
    'Granada': 'Granada CF',
    'Greuther Fürth': 'Greuther Furth',
    'Hellas Verona': 'Verona',
    'Hertha BSC': 'Hertha Berlin',
    'Köln': 'FC Koln',
    'Leeds United': 'Leeds',
    'Leganés': 'Leganes',
    'Leicester City': 'Leicester',
    'Leverkusen': 'Bayer Leverkusen',
    'Luton Town': 'Luton',
    "M'Gladbach": 'B. Monchengladbach',
    'Mainz 05': 'Mainz',
    'Newcastle Utd': 'Newcastle',
    'Milan': 'AC Milan',
    'Norwich City': 'Norwich',
    "Nott'ham Forest": 'Nottingham',
    'Nîmes': 'Nimes',
    'Paderborn 07': 'Paderborn',
    'Paris S-G': 'PSG',
    'Roma': 'AS Roma',
    'SPAL': 'Spal',
    'Saint-Étienne': 'St Etienne',
    'Schalke 04': 'Schalke'
}

UNRESOLVED CONFLICTS FOR TEAMS: 

['Grenoble', 'Hamburger SV', 'Holstein Kiel', 'Paris FC', 'Sochaux'] from scores_df → These teams were in promotion playoffs but somehow mixed with matchday scores. Example is https://www.flashscore.com/football/france/ligue-1-2020-2021/results/ for Grenoble and Paris FC. These results are not within the league and should be filtered

These teams did not make it to the 1st division in any season so are immediately flagged out

In [7]:
teams_df['squad'] = teams_df['squad'].replace(name_mapping)

In [8]:
extra_teams = ['Grenoble', 'Hamburger SV', 'Holstein Kiel', 'Paris FC', 'Sochaux']
scores_df = scores_df[~scores_df['home_team'].isin(extra_teams) & ~scores_df['away_team'].isin(extra_teams)]

In [9]:
null_cols = teams_df.columns[teams_df.isnull().any()]
teams_df.loc[teams_df.isnull().any(axis=1), ['squad', 'season_start_year', *null_cols]]

,squad,season_start_year,gk_pens_save_pct,gk_passes_length_avg,gk_avg_distance_def_actions
8,Liverpool,2018,NaN,28.8,16.6
19,Sheffield Utd,2018,NaN,45.2,10.1
25,Crystal Palace,2019,NaN,48.9,13.2
40,Arsenal,2020,33.3,28.2,NaN
69,Liverpool,2021,NaN,22.7,17.1
102,Atl. Madrid,2018,0.0,33.3,NaN
122,Atl. Madrid,2019,0.0,30.2,NaN
135,Valladolid,2019,12.5,49.5,NaN
200,Atalanta,2018,0.0,50.0,NaN
224,Genoa,2019,0.0,NaN,NaN


In [10]:
print(f"Initial shape: {teams_df.shape}") 
teams_df.drop(columns=null_cols, inplace=True)
teams_df

Initial shape: (488, 185)


,squad,players_used,avg_age,possession,games,games_starts,minutes,minutes_90s,goals,assists,...,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,season_start_year,season_end_year,country
0,Arsenal,28,26.7,58.1,38,418,"3,420",38.0,69,52,...,4,7,1,2000,555,637,46.6,2018,2019,ENG
1,Bournemouth,28,26.6,46.9,38,418,"3,420",38.0,55,43,...,8,5,3,1910,701,736,48.8,2018,2019,ENG
2,Brighton,21,27.4,42.4,38,418,"3,420",38.0,35,24,...,5,10,0,1979,824,780,51.4,2018,2019,ENG
3,Burnley,23,28.0,41.2,38,418,"3,420",38.0,43,32,...,1,3,4,1955,1016,1025,49.8,2018,2019,ENG
4,Chelsea,24,27.3,62.9,38,418,"3,420",38.0,61,52,...,5,2,0,2171,523,504,50.9,2018,2019,ENG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,Rennes,32,25.2,55.4,38,418,"3,420",38.0,66,47,...,3,5,1,2035,538,520,50.9,2022,2023,FRA
484,Strasbourg,29,27.3,45.9,38,418,"3,420",38.0,48,28,...,6,11,0,2132,677,573,54.2,2022,2023,FRA
485,Toulouse,29,25.1,51.3,38,418,"3,420",38.0,50,33,...,2,7,3,2168,483,460,51.2,2022,2023,FRA
486,Le Havre,35,25.2,55.7,38,418,"3,420",38.0,43,32,...,8,1,2,2251,602,542,52.6,2022,2023,FRA


There are other teams who made it to the 1st division in other seasons, who also played matches in playoffs. Their name did not get flagged out previously, but we also want to remove these playoff results

In [11]:
home_scores_df = scores_df[['season_start_year', 'home_team']].rename(columns={'home_team': 'squad', 'season_start_year': 'matches_start_year'})
away_scores_df = scores_df[['season_start_year', 'away_team']].rename(columns={'away_team': 'squad', 'season_start_year': 'matches_start_year'})

combined_scores_df = pd.concat([home_scores_df, away_scores_df], ignore_index=True)
combined_scores_df = combined_scores_df.drop_duplicates(subset=['matches_start_year', 'squad'])

merged_df = pd.merge(combined_scores_df, teams_df, how='outer', left_on=('matches_start_year', 'squad'), right_on=('season_end_year', 'squad'))

null_cols = merged_df.columns[merged_df.isnull().any()]
merged_df.loc[merged_df.isnull().any(axis=1)]

,matches_start_year,squad,players_used,avg_age,possession,games,games_starts,minutes,minutes_90s,goals,...,pens_won,pens_conceded,own_goals,ball_recoveries,aerials_won,aerials_lost,aerials_won_pct,season_start_year,season_end_year,country
318,2019,Heidenheim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,2020,Toulouse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
452,2021,Auxerre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Heidenheim vs Werder Bremen was the playoff in 2019/20 on 03/07 and 07/07
- Auxerre vs St Etienne was the playoff in 2021/22 on 27/05 and 30/05
- Toulouse vs Nantes was the playoff in 2020/21 on 28/05 and 31/05

Hence, all these rows can be dropped

In [12]:
mask_1 = ~(((scores_df['home_team'] == 'Heidenheim') | (scores_df['away_team'] == 'Heidenheim')) & (scores_df['season_start_year'] == 2019))
mask_2 = ~(((scores_df['home_team'] == 'Toulouse') | (scores_df['away_team'] == 'Toulouse')) & (scores_df['season_start_year'] == 2020))
mask_3 = ~(((scores_df['home_team'] == 'Auxerre') | (scores_df['away_team'] == 'Auxerre')) & (scores_df['season_start_year'] == 2021))

print(scores_df.shape)
scores_df = scores_df[mask_1 & mask_2 & mask_3]
print(scores_df.shape)
display(scores_df)

(8961, 13)
(8955, 13)


,time,home_team,home_score,away_team,away_score,season_start_year,season_end_year,year,weekday,month,day,hour,minute
0,2019-06-18 01:00:00,Aston Villa,0.0,Sheffield Utd,0.0,2019,2020,2019,1,6,18,1,0
1,2019-06-18 03:15:00,Manchester City,3.0,Arsenal,0.0,2019,2020,2019,1,6,18,3,15
2,2019-06-20 01:00:00,Norwich,0.0,Southampton,3.0,2019,2020,2019,3,6,20,1,0
3,2019-06-20 03:15:00,Tottenham,1.0,Manchester Utd,1.0,2019,2020,2019,3,6,20,3,15
4,2019-06-20 19:30:00,Watford,1.0,Leicester,1.0,2019,2020,2019,3,6,20,19,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8966,2024-05-19 03:00:00,Lorient,NaN,Clermont,NaN,2023,2024,2024,6,5,19,3,0
8967,2024-05-19 03:00:00,Lyon,NaN,Strasbourg,NaN,2023,2024,2024,6,5,19,3,0
8968,2024-05-19 03:00:00,Metz,NaN,PSG,NaN,2023,2024,2024,6,5,19,3,0
8969,2024-05-19 03:00:00,Monaco,NaN,Nantes,NaN,2023,2024,2024,6,5,19,3,0


Get form for last 5 games

In [13]:
def get_points_from_game(row, team_name):    
    is_home = row["home_team"] == team_name
    side = "home" if is_home else "away"
    opp_side = "away" if is_home else "home"
    return (3 if row[f"{side}_score"] > row[f"{opp_side}_score"] else
        1 if row[f"{side}_score"] == row[f"{opp_side}_score"] else 0)
    
def get_teams_form(row, depth=5):
    """    Returns points per game from past :depth: games (np.nan if no history) for ONE team
    """    
    form = {
        'home': np.nan,
        'away': np.nan
    }
    for side in ['home', 'away']:
        previous_games_df = scores_df[
            (scores_df["time"] < row["time"]) & (
                (row[f"{side}_team"] == scores_df["home_team"]) | (row[f"{side}_team"] == scores_df["away_team"])
            )]
        if len(previous_games_df) > 0:        
            form[side] = (
                previous_games_df.sort_values(by="time", ascending=False).head(depth)
                .apply(lambda x: get_points_from_game(x, row[f"{side}_team"]), axis=1).mean()
            )
    return form
    
def get_form(row, depth=5):
    """    Returns points per game from past :depth: games (np.nan if no history) for BOTH teams
    """    
    return get_teams_form(row, depth).values()

scores_df[["home_team_form", "away_team_form"]] = scores_df.apply(get_form, axis=1, result_type='expand')

# Creating Results Dataframe

In [14]:
# Results DF table
HOME = "home"
AWAY = "away"
SIDES = (HOME, AWAY)

# Create new dataframe with all unique team names
results_df = pd.DataFrame(
    set(scores_df["home_team"].unique()).union(set(scores_df["away_team"].unique())),
    columns=["team_name"]
).sort_values(by=["team_name"]).reset_index(drop=True)

results_df = results_df.rename(columns={"team_name": "squad"})

# Initialise empty columns to 0
results_df_cols = ["points", "goals_for", "goals_against"]
results_df_cols += [f"{col}_{side}" for side in SIDES for col in results_df_cols]
for col in ["season_start_year", "season_end_year", *results_df_cols]:
    results_df[col] = 0

for idx, row in scores_df.iterrows():
    for side in SIDES:
#         Find correct year and team in results dataframe -> get index to modify row with .loc[]
        target = results_df[
            (results_df["season_start_year"] == row["season_start_year"]) &
            (results_df["season_end_year"] == row["season_end_year"]) &
            (results_df["squad"] == row[f"{side}_team"])
        ]
        if target.empty:
            results_df = pd.concat([results_df, pd.DataFrame([[
                row[f"{side}_team"], row["season_start_year"],
                row["season_end_year"], *[0 for _ in range(len(results_df.columns)-3)]
            ]], columns=results_df.columns)], ignore_index=True)
            target_idx = len(results_df)-1
        else:
            target_idx = target.index
        
#         Add stats to relevant column (home/away team)
        opp_side = HOME if side == AWAY else AWAY
        results_df.loc[target_idx, ["points", f"points_{side}"]] += (
            3 if row[f"{side}_score"] > row[f"{opp_side}_score"] else
            1 if row[f"{side}_score"] == row[f"{opp_side}_score"] else 0
        )
        results_df.loc[target_idx, ["goals_for", f"goals_for_{side}"]] += row[f"{side}_score"]
        results_df.loc[target_idx, ["goals_against", f"goals_against_{side}"]] += row[f"{opp_side}_score"]

# Drop rows initialised to 0
results_df = results_df[results_df["season_start_year"] != 0].reset_index(drop=True)

# Creating Squad Value Dataframe

In [15]:
squad_value_dfs = {}

for country in COUNTRIES:
    for year in range(2018, 2024):
        for tier in range(1, 3):
            try:
                squad_value_dfs[f"{year}_{country}{tier}"] = pd.read_csv(
                    f"./data/squad_values/raw/{country}{tier}{year}_squad_values_{year+1}.csv",
                    encoding='utf-8'
                )
                squad_value_dfs[f"{year}_{country}{tier}"][["season_start_year", "season_end_year"]] = [year, year+1]
            except FileNotFoundError as e:
                print(e)
raw_squad_value_df = pd.concat(squad_value_dfs.values(), ignore_index=True)
raw_squad_value_df["squad"] = raw_squad_value_df["squad"].replace(
    dict(zip(SQUAD_NAME_MAPPING["transfermarkt"], SQUAD_NAME_MAPPING["final"])),
    regex=True
)

pkey = ['squad', "season_start_year", "season_end_year"]
teams_df = pd.merge(
    teams_df,
    raw_squad_value_df,
    how='left',
    left_on=pkey,
    right_on=pkey,
)

# Creating Aggregated/per90 Dataframe

This dataframe will be used for EDA and machine learning

`agg_teams_df` aggregates the dataframe into per 90 stats instead of actual stats over the whole season so that it is more accurate



In [21]:
# Drop unwanted columns
teams_agg_df = teams_df[list(sum(TEAM_COLUMNS_DICT_COMBINED.values(), []))].copy()

teams_agg_df["gd_per90"] = teams_agg_df["goals_per90"] - teams_agg_df["gk_goals_against_per90"]
TEAM_COLUMNS_DICT['squad']['aggregated'].append('gd_per90')

FULL_AGG_COLUMNS_DICT = {}
for cat, col_types in TEAM_COLUMNS_DICT.items():
    new_cols = [f"{col}_per90" for col in col_types["unaggregated"]]
    teams_agg_df[new_cols] = teams_agg_df[col_types["unaggregated"]].div(teams_agg_df['minutes_90s'], axis=0)
    agg_cols = col_types["aggregated"] + new_cols
    FULL_AGG_COLUMNS_DICT[cat] = agg_cols

AGG_COL_TO_CAT_DICT = {col: cat for (cat, cols) in FULL_AGG_COLUMNS_DICT.items() for col in cols}

teams_agg_df = teams_agg_df[AGG_COL_TO_CAT_DICT.keys()]

In [22]:
teams_agg_df[['season_start_year', 'season_end_year']]

,season_start_year,season_end_year
0,2018,2019
1,2018,2019
2,2018,2019
3,2018,2019
4,2018,2019
...,...,...
483,2022,2023
484,2022,2023
485,2022,2023
486,2022,2023


# Creating EDA Dataframe

`eda_df` will use per90 stats as well but it will merge with the current season's results so that we can find correlations between performance and results over one season itself.

In [24]:
# Merge with results dataframe
eda_df = pd.merge(
    results_df, # drop to not confuse which end year is correct...
    teams_agg_df,
    how='inner',
    left_on=['squad', "season_start_year", "season_end_year"],
    right_on=['squad', "season_start_year", "season_end_year"]
)

for cat, col_types in SCORES_COLUMNS_DICT.items():
    new_cols = [f"{col}_per90" for col in col_types["unaggregated"]]
    eda_df[new_cols] = eda_df[col_types["unaggregated"]].div(eda_df['minutes_90s'], axis=0)
    agg_cols = col_types["aggregated"] + new_cols
    if cat not in FULL_AGG_COLUMNS_DICT:
        FULL_AGG_COLUMNS_DICT[cat] = agg_cols
    else:
        FULL_AGG_COLUMNS_DICT[cat] = list(set(FULL_AGG_COLUMNS_DICT[cat]) | set(agg_cols))

AGG_COL_TO_CAT_DICT = {col: cat for (cat, cols) in FULL_AGG_COLUMNS_DICT.items() for col in cols}

# Get per-game stats
# Note: meta fields like squad not inclued in unaggregated_cols
eda_df = eda_df[AGG_COL_TO_CAT_DICT.keys()]

These dictionaries may be useful in future so we save it

In [25]:
with open(FULL_AGG_COLUMNS_DICT_PATH, 'w') as file:
    json.dump(FULL_AGG_COLUMNS_DICT, file, indent=4)
with open(AGG_COL_TO_CAT_DICT_PATH, 'w') as file:
    json.dump(AGG_COL_TO_CAT_DICT, file, indent=4)

In [26]:
eda_df[['goals_per90', 'goals_for_home_per90', 'goals_for_away_per90']]

,goals_per90,goals_for_home_per90,goals_for_away_per90
0,1.05,0.578947,0.500000
1,0.95,0.631579,0.394737
2,2.63,1.500000,1.184211
3,1.47,0.947368,0.526316
4,1.34,0.552632,0.789474
...,...,...,...
332,2.26,1.184211,1.157895
333,1.16,0.526316,0.657895
334,1.18,0.736842,0.447368
335,1.68,0.763158,0.947368


Save these CSVs before we merge all together and drop `season_start_year` for teams_agg and `season_end_year` for scores (see below)

In [42]:
scores_df.to_csv("./data/scores/scores.csv", index=False)
results_df.to_csv("./data/scores/results.csv", index=False)
raw_squad_value_df.to_csv("./data/squad_values/squad_values.csv", index=False)

# teams_df contains data without per90 values
teams_df.to_csv("./data/teams/teams.csv", index=False)

# per90_df contains only aggregated data
eda_df.to_csv("./data/exploratory_analysis/eda.csv", index=False)

# Complete DF is used for Exploratory Analysis
teams_agg_df.to_csv("./data/teams/teams_agg.csv", index=False)

# Lastly, create the win_lose dataframe for machine learning

We are merging the end year performance of the previous season on the start year of the scores... 
```
end_2022_teams_agg_data --- predicts ----> start_2023_scores_data
```
So the start year of performance data `teams_agg_df` can be dropped and end year of the scores data `scores_df` can be dropped

In [29]:
teams_agg_df = teams_agg_df.drop(['season_start_year'], axis=1)
scores_df = scores_df.drop(['season_end_year'], axis=1)

In our current dataset, each football match is represented as a single row with separate columns for the home team and the away team. This format is intuitive for representing matches, but for our analysis, we want to focus on the performance of individual teams, not just matches.

To achieve this, we create a new structure where for every match, we have two entries; one for each team involved. In each entry, the team we're focusing on will be listed in a new 'team' column, and their opponent in that match will be in an 'opponent' column.

This dual perspective ensures that every team's activity, whether they are playing at home or away, gets equal representation in our analysis, providing us with a more balanced and comprehensive view of their performance throughout the season.

The current scores table:
<table>
    <thead>
        <tr>
            <th>home_team</th>
            <th>away_team</th>
            <th>home_score</th>
            <th>away_score</th>
            <th>home_team_form</th>
            <th>away_team_form</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>teamA</td>
            <td>teamB</td>
            <td>3</td>
            <td>1</td>
            <td>1.5</td>
            <td>0.5</td>
        </tr>
    </tbody>
</table>

The new scores dataframe
<table>
    <thead>
        <tr>
            <th>team</th>
            <th>opponent</th>
            <th>is_home</th>
            <th>score</th>
            <th>opponent_score</th>
            <th>form</th>
            <th>opponent_form</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>teamA</td>
            <td>teamB</td>
            <td>1</td>
            <td>3</td>
            <td>1</td>
            <td>1.5</td>
            <td>0.5</td>
        </tr>
        <tr>
            <td>teamB</td>
            <td>teamA</td>
            <td>0</td>
            <td>1</td>
            <td>3</td>
            <td>0.5</td>
            <td>1.5</td>
        </tr>
    </tbody>
</table>

In [31]:
home_df = scores_df.copy()
print(home_df['home_team_form'].isna().sum())
away_df = scores_df.copy()

home_df['team'] = home_df['home_team']
home_df['opponent'] = home_df['away_team']
home_df['is_home'] = 1
home_df['score'] = home_df['home_score']
home_df['opponent_score'] = home_df['away_score']
home_df['form'] = home_df['home_team_form']
home_df['opponent_form'] = home_df['away_team_form']

print(home_df['form'].isna().sum())

away_df['team'] = away_df['away_team']
away_df['opponent'] = away_df['home_team']
away_df['is_home'] = 0
away_df['score'] = away_df['away_score']
away_df['opponent_score'] = away_df['home_score']
away_df['form'] = away_df['away_team_form']
away_df['opponent_form'] = away_df['home_team_form']

old_cols = ['time', 'home_team', 'away_team', 'home_score', 'away_score', 'home_team_form', 'away_team_form']

final_scores_df = pd.concat([home_df, away_df], ignore_index=True)
final_scores_df = final_scores_df.drop(old_cols, axis=1)
print(home_df['form'].isna().sum())
final_scores_df

63
63
63


,season_start_year,year,weekday,month,day,hour,minute,team,opponent,is_home,score,opponent_score,form,opponent_form
0,2019,2019,1,6,18,1,0,Aston Villa,Sheffield Utd,1,0.0,0.0,NaN,NaN
1,2019,2019,1,6,18,3,15,Manchester City,Arsenal,1,3.0,0.0,NaN,NaN
2,2019,2019,3,6,20,1,0,Norwich,Southampton,1,0.0,3.0,NaN,NaN
3,2019,2019,3,6,20,3,15,Tottenham,Manchester Utd,1,1.0,1.0,NaN,NaN
4,2019,2019,3,6,20,19,30,Watford,Leicester,1,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17905,2023,2024,6,5,19,3,0,Clermont,Lorient,0,NaN,NaN,0.2,0.0
17906,2023,2024,6,5,19,3,0,Strasbourg,Lyon,0,NaN,NaN,0.6,0.6
17907,2023,2024,6,5,19,3,0,PSG,Metz,0,NaN,NaN,0.0,0.6
17908,2023,2024,6,5,19,3,0,Nantes,Monaco,0,NaN,NaN,0.6,0.0


Next, we want to merge the scores on the teams dataframe to get the following table 

<table>
    <thead>
        <tr>
            <th>team</th>
            <th>opponent</th>
            <th>is_home</th>
            <th>score</th>
            <th>opponent_score</th>
            <th>form</th>
            <th>opponent_form</th>
            <th>team_stat_1</th>
            <th>team_stat_2</th>
            <th>opponent_stat_1</th>
            <th>opponent_stat_2</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>teamA</td>
            <td>teamB</td>
            <td>1</td>
            <td>3</td>
            <td>1</td>
            <td>1.5</td>
            <td>0.5</td>
            <td>1</td>
            <td>2</td>
            <td>3</td>
            <td>4</td>
        </tr>
        <tr>
            <td>teamB</td>
            <td>teamA</td>
            <td>0</td>
            <td>1</td>
            <td>3</td>
            <td>0.5</td>
            <td>1.5</td>
            <td>3</td>
            <td>4</td>
            <td>1</td>
            <td>2</td>
        </tr>
    </tbody>
</table>

Note that we merge the `teams_df` on the **previous season** (end year) with the `scores_df` of the **current season** (start year). Once again this is so that we can predict the scores for the current season using the teams' performances of the previous season.

Also, only `season_start_year` remains after this, `season_end_year` has been dropped. `season_start_year` is the year of the results

In [32]:
teams_df = teams_agg_df.add_prefix('team_')
teams_vs_df = teams_agg_df.add_prefix('opponent_')

merged_df = pd.merge(
        final_scores_df, teams_df, how='left', left_on=['team', 'season_start_year'], right_on=['team_squad', 'team_season_end_year']
    )
merged_df = merged_df.drop(['team_squad', 'team_season_end_year'], axis=1)

merged_df = pd.merge(
        merged_df, teams_vs_df, how='left', left_on=['opponent', 'season_start_year'], right_on=['opponent_squad', 'opponent_season_end_year']
    )
merged_df = merged_df.drop(['opponent_squad', 'opponent_season_end_year'], axis=1)

print(f'Original Performance DF has {len(teams_agg_df.columns)} columns')
print(f'Original Scores DF has {len(final_scores_df.columns)} columns')
print(f'Final Dataframe should have {len(teams_agg_df.columns) * 2 + len(final_scores_df.columns) - 4} columns')
print(f'Final Dataframe has {len(merged_df.columns)} columns')

Original Performance DF has 129 columns
Original Scores DF has 14 columns
Final Dataframe should have 268 columns
Final Dataframe has 268 columns


For our **test and train** data, we will use the scores up till **2022-23** season.

Our plan is to eventually use the models we have refined to **predict** scores for the **2023-24** season, so for that season, we will drop the `scores` and `form`.

Moreover, the file is too big to be push to github. So we split the data based on year

Before that, we create another column to determine the result 'W', 'D', or 'L', so that instead of predicting 2 labels `score` and `opponent_score`, some models can predict `result` instead

Add the results ('W', 'D', or 'L') into a 'result' column

In [33]:
conditions = [
    merged_df['score'] > merged_df['opponent_score'],  # Win
    merged_df['score'] == merged_df['opponent_score'], # Draw
    merged_df['score'] < merged_df['opponent_score']   # Loss
]
choices = ['W', 'D', 'L']
merged_df['outcome'] = np.select(conditions, choices)

In [34]:
# save the files
for year in range(2019, 2024):
    df = merged_df[merged_df['season_start_year'] == year]
    if year == 2023:
        df = df.drop(['score', 'opponent_score', 'outcome', 'form', 'opponent_form', ], axis=1)
        df.to_csv(f"./data/machine_learning/{str(year)}_predict.csv", index=False)
    else:
        one_label_df = df.drop(['score', 'opponent_score'], axis=1)
        one_label_df.to_csv(f"./data/machine_learning/outcome/{str(year)}.csv", index=False)
        two_label_df = df.drop(['outcome'], axis=1)
        two_label_df.to_csv(f"./data/machine_learning/scoreline/{str(year)}.csv", index=False)

Check that for the null values in the rows are only due to missing data for form (for games at the start of every season)

In [35]:
cur_season_mask = merged_df['season_start_year'] == 2023
bef_cur_season_mask = ~cur_season_mask
bef_2023_df = merged_df[bef_cur_season_mask]

In [36]:
rows_with_na = bef_2023_df[['form', 'opponent_form']].isna().any(axis=1).sum()
rows_with_na

170

In [38]:
# All null data

null_cols = bef_2023_df.columns[bef_2023_df.isnull().any()]
bef_2023_df.loc[bef_2023_df.isnull().any(axis=1), ['team', 'is_home', 'season_start_year', *null_cols]]

,team,is_home,season_start_year,form,opponent_form
0,Aston Villa,1,2019,NaN,NaN
1,Manchester City,1,2019,NaN,NaN
2,Norwich,1,2019,NaN,NaN
3,Tottenham,1,2019,NaN,NaN
4,Watford,1,2019,NaN,NaN
...,...,...,...,...,...
16470,Lens,0,2020,NaN,1.6
16846,PSG,0,2021,2.6,NaN
16849,Clermont,0,2021,NaN,1.8
17224,Marseille,0,2022,1.8,NaN


In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Simulated data
data = {
    'matchday': [1, 2, 3, 4],
    'team_form': [50, None, None, None],  # Initial form known, others to be predicted
    'result': [1, None, None, None]  # Only the result of the first match is known
}
df = pd.DataFrame(data)

# Assume team_form updates based on whether they won the last match (+10 for a win, -10 for a loss)

# Initial training data (usually you'd have much more data)
X_train = pd.DataFrame({
    'team_form': [40, 50, 60],  # Historical forms
    'result': [0, 1, 1]  # Historical results
})
y_train = [0, 1, 1]  # Outcomes of the matches

# Create and train the RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train[['team_form']], y_train)

for i in range(1, len(df)):
    # Update the team_form based on the last match's result
    last_result = df.at[i-1, 'result']
    last_form = df.at[i-1, 'team_form']
    if last_result == 1:
        new_form = last_form + 10
    else:
        new_form = last_form - 10

    # Predict the result of the current matchday using the updated form
    predicted_result = model.predict([[new_form]])[0]
    df.at[i, 'team_form'] = new_form
    df.at[i, 'result'] = predicted_result

print(df)

   matchday  team_form  result
0         1       50.0     1.0
1         2       60.0     1.0
2         3       70.0     1.0
3         4       80.0     1.0


/home/rachtrx/mambaforge/envs/sc1015/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/rachtrx/mambaforge/envs/sc1015/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/home/rachtrx/mambaforge/envs/sc1015/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
